In [140]:
# Import packages
import numpy as np
curDay = 19

# Download input file
from download_inputs import download_inputs


from tqdm import tqdm


import os

filename = f'inputs\\day{curDay}.txt'
if not os.path.exists(filename):
    download_inputs(curDay)
else:
    print('Inputs already downloaded')

Inputs already downloaded


In [141]:
# Load input into workspace
filename = f'inputs\\day{curDay}.txt'

inputFile = open(filename)
inputs = [s for s in inputFile.read().splitlines()]

print('Done loading input')

Done loading input


In [142]:
# Load test-input into workspace
filenameTest = f'inputs\\day{curDay}_test.txt'

inputFileTest = open(filenameTest)
inputsTest = [s for s in inputFileTest.read().splitlines()]

print('Done loading test-input') 

Done loading test-input


# Input parsing

In [143]:
def parseInputs(test=False):
    if test:
        curIn = inputsTest 
    else:
        curIn = inputs 
        

    liW = []
    liP = []
    inW = True 
    for li in curIn:
        if inW:
            liW.append(li)
        else:
            liP.append(li)
        
        if li == '':
            inW = False

    liW = liW[:-1]


    # liW,liP

    # curP = liP[0]

    pDict = []
    for curP in liP:
        # newDict = dict([x.split('=') for x in curP[1:-1].split(',')])
        curKeys = [x.split('=')[0] for x in curP[1:-1].split(',')]
        curVals = [int(x.split('=')[1]) for x in curP[1:-1].split(',')]
        newDict = dict(zip(curKeys,curVals))
        # print([x.split('=')[0] for x in curP[1:-1].split(',')])
        # print([int(x.split('=')[1]) for x in curP[1:-1].split(',')])

        pDict.append(newDict)

    # curW = liW[0]
    wDict = []
    wDict = dict()
    for curW in liW:
        curName,curRules = curW[:-1].split('{')
        curRules = curRules.split(',')
        

        wDict[curName] = curRules
        
    return wDict,pDict  

# Part 1

In [144]:
wDict,pDict = parseInputs(test=True)
wDict,pDict = parseInputs(test=False)


In [145]:

def checkPartInWorkflow(curP,curW):
    for rule in wDict[curW]:
        # print(rule) 
        # print(rule.split(':'))
        ruleSplit = rule.split(':')
        if len(ruleSplit) > 1:
            # print(ruleSplit)
            condi,posGoal = ruleSplit 
            # print(condi)

            if '<' in condi:
                cate,compVal = condi.split('<')
                compVal = int(compVal)
                # print(cate,'less than',compVal)
                if (curP[cate] < compVal):
                    # print('Success')
                    newW = posGoal 
                    break 
            elif '>' in condi:
                cate,compVal = condi.split('>')
                compVal = int(compVal)
                # print(cate,'greater than',compVal)
                if (curP[cate] > compVal):
                    # print('Success')
                    newW = posGoal 
                    break 
            else: 
                print('Something is wrong, should not happen')
        else:
            newW = rule 
    # print(newW)
    return newW 



In [211]:
wDict,pDict = parseInputs(test=False)


In [212]:
curP = pDict[0]
curW = 'in'

isAcc = []
sol1 = 0

for curID in range(len(pDict)):
    curP = pDict[curID]
    curW = 'in'

    notDone = True 
    while notDone:


        curW = checkPartInWorkflow(curP,curW)
        # print(f'{curW=}')
        if curW == 'A':
            isAcc.append(True)

            sol1 += np.sum([*curP.values()])
            notDone = False 
        if curW == 'R':
            isAcc.append(False)
            notDone = False 


In [213]:
print(f'{sol1 = }')

sol1 = 397061


# Part 2


## splitting of 4D space, recursively

In [199]:
wDict,pDict = parseInputs(test=False)
# wDict,pDict = parseInputs(test=True)

In [200]:
def calcSpaceSize(curSpace):
    rangeX = curSpace['x'][1] - curSpace['x'][0] + 1
    rangeM = curSpace['m'][1] - curSpace['m'][0] + 1
    rangeA = curSpace['a'][1] - curSpace['a'][0] + 1
    rangeS = curSpace['s'][1] - curSpace['s'][0] + 1

    return rangeX*rangeM*rangeA*rangeS 

In [208]:

global sol2 
sol2 = 0

def recuFunc(curSpace,workflowName,ruleID):
    global sol2
    # print('---',workflowName,ruleID,'---')
    curW = wDict[workflowName]
    curR = curW[ruleID]

    ruleSplit =curR.split(':')    
    if len(ruleSplit) > 1:
        condi,posGoal = ruleSplit 

    

        if '<' in condi:
            cate,compVal = condi.split('<')
            compVal = int(compVal)

            SpaceToCheck = curSpace[cate]

            ### Recursively check sub-spaces
            # print(cate,SpaceToCheck)
            # In case current maximum possible is already below, no split
            if SpaceToCheck[1] < compVal:
                print('-Is already below, moves to subspace')

                newSpace = curSpace.copy()
                ### Recursion here (?)
                recuFunc(newSpace,posGoal,0)


            else:
                # Split in two
                newMin1 = SpaceToCheck[0]
                newMax1 = compVal-1
                newMin2 = compVal
                newMax2 = SpaceToCheck[1]

                
                newSpace1 = curSpace.copy()
                newSpace2 = curSpace.copy()
                newSpace1[cate] = [newMin1,newMax1]
                newSpace2[cate] = [newMin2,newMax2]
                newRuleID = ruleID + 1

                # # print(newMin1,newMax1,newMin2,newMax2)
                # print(f'{newSpace1 = }')
                # print(f'Moves to {posGoal}, 0')
                # print(f'{newSpace2 = }')
                # print(f'Stays in {workflowName}, moves to {newRuleID}')
                # Number one continue down tree, number two moves to next rule on current branch
                ### RECURSION HERE (?)
                if (posGoal != 'A') & (posGoal != 'R'):
                    recuFunc(newSpace1,posGoal,0)
                else:
                    # print('Finished',newSpace1,posGoal)
                    if posGoal == 'A':
                        sol2 += calcSpaceSize(newSpace1)
                        # print('Space: ',calcSpaceSize(newSpace1))
                
                
                recuFunc(newSpace2,workflowName,newRuleID)




        elif '>' in condi:
            cate,compVal = condi.split('>')
            compVal = int(compVal)

            SpaceToCheck = curSpace[cate]

            ### Recursively check sub-spaces
            # print(cate,SpaceToCheck)
            # In case current minimum possible is already above, no split
            if SpaceToCheck[0] > compVal:
                print('-Is already above, moves to branch')

                newSpace = curSpace.copy()
                ### Recursion here (?)
                recuFunc(newSpace,posGoal,0)
            else:
                # Split in two, one stays, one leaves
                newMin1 = SpaceToCheck[0]
                newMax1 = compVal
                newMin2 = compVal+1
                newMax2 = SpaceToCheck[1]

                
                newSpace1 = curSpace.copy()
                newSpace2 = curSpace.copy()
                newSpace1[cate] = [newMin1,newMax1]
                newSpace2[cate] = [newMin2,newMax2]
                newRuleID = ruleID + 1

                # print(newMin1,newMax1,newMin2,newMax2)
                # print(f'{newSpace1 = }')
                # print(f'Stays in {workflowName}, moves to {newRuleID}')
                # # print(f'Moves to {posGoal}, 0')
                # print(f'{newSpace2 = }')
                # # print(f'Stays in {workflowName}, moves to {newRuleID}')
                # print(f'Moves to {posGoal}, 0')
                
                if (posGoal != 'A') & (posGoal != 'R'):
                    recuFunc(newSpace2,posGoal,0)
                else:
                    # print('Finished',newSpace2,posGoal)
                    if posGoal == 'A':
                        sol2 += calcSpaceSize(newSpace2)
                        # print('Space: ',calcSpaceSize(newSpace2))
                
                recuFunc(newSpace1,workflowName,newRuleID)
                


            
        else: 
            print('ERROR!')

    else:
        # No comparison, i.e. final possible outcome, so simply go to branch
        # print('Last option, move to branch')
        
        newSpace = curSpace.copy()
        ### Recursion here (?)
        
        if (curR != 'A') & (curR != 'R'):
            recuFunc(newSpace,curR,0)
        else:
            # print('Finished',newSpace,curR)
            if curR == 'A':
                sol2 += calcSpaceSize(newSpace)
                # print('Space: ',calcSpaceSize(newSpace))
        # recuFunc(newSpace,curR,0)


In [209]:
        
curSpace = {
    'x':[1,4000],
    'm':[1,4000],
    'a':[1,4000],
    's':[1,4000],
}

# print(wDict['crn'])
recuFunc(curSpace,'in',0)


In [210]:
print(f'{sol2 = }')

sol2 = 125657431183201


# Old stuff, branching thing for part 2

Original idea: build a branching tree of all possibilities, find some way to count it afterwards


In [ ]:
def getAllBranches(name):
    curPos = wDict[name]

    # for i in range(len(curPos)-1):
    # print(curPos)
    toReturn = []
    for br in curPos:
        # print(br)
        curBr = br.split(':')[-1]
        # print(curBr)
        
        if (curBr == 'A') | (curBr == 'R'):
            toReturn.append(curBr)
        else:
            lowerBranches = getAllBranches(curBr)
            toReturn.append(lowerBranches)
            # print(lowerBranches)
        # # toReturn.append(curBr)
        # if (curBr != 'A') & (curBr != 'R'):
        #     lowerBranches = getAllBranches(curBr)
        #     toReturn.append(lowerBranches)
        # else:
        #     toReturn.append(curBr)
            # toReturn.append([curBr,])
    # print(toReturn)
    return {name:toReturn}
        # print(br.split(':')[-1])

allBr = getAllBranches('in')
allBr
# # getAllBranches('crn')
# thisBr = getAllBranches('qkq')
# # thisBr['qkq'][1]
# # wDict['qkq']